---
>「ドライバーが車を選ぶんじゃない。車がドライバーを選ぶんだ。人間と機械の間には、神秘的な絆があるんだ」 \
>（ボビー:映画トランスフォーマーより）
---

# Transformerモデルを用いた文章分類

シンプルなクラス分類のTransformerモデルをフルスクラッチで実装する
- 映画の英語レビューがポジテイプな内容かネガテイプな内容かを判定させる
- どのような単語に注目して判定したのかをSelf-Attentionの結果から可視化する

**<font color="red">+++注意+++</font>**

ライブラリの更新が速いため、バージョン違いによる動作エラーが発生する場合があります
- 現在把握している問題については、回避方法を記述していますが、再起動が必要となるなど、単純に実行しただけでは結果を得ることができない場合があります

## なぜTransformerが重要なのか

Transformerは、自然言語処理に適する方法として見出されたが、これにはどのような経緯があったのか、簡単に説明する


### 自然言語処理はなぜ困難なのか

まず、言語データは画像データと本質的に扱い方が異なる

- 画像データは画素の集まりであり、CNNでは画像全体から近隣の画素の特徴をとらえて処理する
- 言語データは単語を逐次的に聞いて処理する
  - つまり、過去の単語の入力情報を保持し、文脈を理解する必要がある
  - 例えば、「まいった」だけではわからないが、「失敗してまいった」「神社にまいった」となればわかる

言語データの性質から、逐次的に処理する仕組みが必要
- 従来は、この性質から、RNN (Recurrent Neural Network)やLSTM (Long short-term memory)などが利用されてきた
  - 内部状態として過去の状態を記録することができるため
- 一方で、RNN、LSTMは学習時間が長く大きなモデルを構築するのが困難であった
  - 文章の単語を1stepに1単語ずつモデルに投入するため、バッチにより並列的に大量に処理できるCNNと異なり時間がかかる

処理速度を稼ぐため、CNNやFCを利用する試みもなされた
- 処理速度は向上するが、やはり文章の離れた単語間の関係性を考慮できないため、精度の向上が困難であった
  - 主語と述語は文章の最初と最後であり、この主述関係を理解しようとするとCNNや勾配消失の大きいRNNでは困難であることは容易に想像できる

### Attentionの登場

既に説明済みであるがTransformerではAttentionという新しい概念が提案、利用されている

- Attentionの演算は内積演算であり、同じ向きを向いた大きな要素間では大きな値となるため、これを類似度として扱う
  - これを値の類似度としてとらえるには抵抗を感じるかもしれないが、ひとまずここではこのようにして計算した値を類似度とする
- この掛け算の値をAttention Mapと呼ぶ

- Self-AttentionおよびSource-Target-Attentionを利用することで、言語データでも離れた位置の単語間の関係を考慮できる可能性がある

## 事前準備

今回はフルスクラッチで記述するため、シンプルである
- このあと機能に特化したライブラリは個別に読み込んでいる
- なお、PyTorchのTransformerライブラリを用いるなどして、個別モジュールの設計を避けてパーツを組み合わせることで実装することを推奨する
- 提供されるライブラリは下記の記述よりも実行速度が速い、最適化されている、より優れた実装が採用されている、なによりも精度が高くなるなど、良いことばかりであり、そもそも利用するという立場では一から設計する意味はほとんどない

In [ ]:
import math
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchtext
from torch.utils.data import DataLoader
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

nlpライブラリは様々なデータセットを提供している
- 映画評論データセットを入手するために利用する

In [ ]:
!pip install nlp
!pip install --force-reinstall dill==0.3.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.7
    Uninstalling dill-0.3.7:
      Successfully uninstalled dill-0.3.7


transformersライブラリを読み込んでいるが、transformerのモデルを利用するわけではない
- ここでは、AutoTokenizerを利用するために読み込んでいる
- AutoTokenizerはHuggingfaceが提供している有用性の高いライブラリであり、今後主流となる予感がする
- AutoTokenizerはDataLoaderとの相性がよいため安心して利用できる
  - `BertForSequenceClassification.from_pretrained`などを用いることもできるが、バッチ処理がかなり面倒になるであろう
  - BertTokenizerFastに引けをとらない処理速度を有している

**重要な点**
今回は学習させるため、どのようなTokenizerを用いても構わない
- なんなら自作でも構わない

事前学習済みモデルを利用する場合は、そのモデルが用いたTokenizerを用いなければ正しい結果を得ることができない
- 当然であるが、「私」を10に変換していたのが、変わって20に変換されては精度が落ちて当然
- 危険なのは、この件に限らず、間違えても頑張って学習する結果、それなりに精度が出るため、「誤りを、誤りと気づきにくい」点に注意が必要である

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.6 MB/s eta 0:00:00


### モデルとTokenizerの読み込み
事前学習済みのモデルと、これと紐づいたTokenizerを読み込む
- いつもと同様、bert-base-uncased 事前学習モデルを読み込む

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

### データセットの読み込み

nlpライブラリに含まれるIMDbデータセットを利用する
- IMDbデータセットは、ポジティブかネガティブの好悪感情を表すラベルが付与された25000の映画レビューコメントデータセット
- 好意的なレビューは1、否定的なレビューは0が振られている
- 感情分析用では鉄板のデータセット

https://www.imdb.com/interfaces/

In [ ]:
from nlp import load_dataset
raw_train_data, raw_test_data = load_dataset("imdb", split=["train", "test"]) # 訓練用と検証用データに分けて読み込む

Downloading:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/76cdbd7249ea3548c928bbf304258dab44d09cd3638d9da8d42480d1d1be3743. Subsequent calls will reuse this data.


例としてデータを表示する
- 英語です、がっかりしましたか？

In [ ]:
print(raw_train_data["label"][0], raw_train_data["text"][0])  # 好意的なコメントの例
print(raw_train_data["label"][20000], raw_train_data["text"][20000])  # 否定的なコメントの例

1 Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!
0 This movie tries hard, but completely lacks the fun of the 1960s TV series, that I am sure people do remember with fondness. Although I am 17, I watched some of the series on YouTube a long

DeepLで訳してみると次のような感じです

> 1 ブロムウェル・ハイ」は、カートゥーン・コメディです。ブロムウェル・ハイ』は、『ティーチャーズ』のような学校生活を描いた番組と同時期に放送されていました。私の35年間の教師生活を振り返ると、「ブロムウェル・ハイ」の風刺は「ティーチャーズ」よりもはるかに現実に近いものだと思います。経済的に生き残るために奔走する姿、哀れな教師たちの虚勢を見抜く洞察力のある生徒たち、そしてすべての状況の情けなさは、私が知っている学校とその生徒たちを思い出させてくれます。生徒が何度も学校を燃やそうとしたエピソードを見たとき、すぐに ......... .......... のことを思い出しました。高いですね。古典的なセリフです。検閲官：あなた方の先生の一人をクビにするために来ました。生徒：Bromwell Highへようこそ。私と同年代の大人の多くは、「ブロムウェルハイ」を奇想天外なものだと思っているのではないでしょうか。そうでないのが残念です。

> 0 この映画は努力していますが、1960年代のテレビシリーズの面白さが完全に欠けています。私は17歳ですが、ずいぶん前にYouTubeでこのシリーズを見たことがあり、楽しくて仕方がありませんでした。特殊効果は標準的ではなく、平板なカメラワークによって助けられていませんでした。また、「ホームアローン4」、「帽子をかぶった猫」、「きかんしゃトーマス」、「アダムス・ファミリー・リユニオン」などの作品があります。さて、ストーリーのアイデアは良かったのですが、残念ながら出来が悪く、早々に力尽きてしまったので、正直、家族で楽しめる作品ではないと思います。また、ウェイン・ナイトが気合を入れて演じたにもかかわらず、しゃべるスーツにも腹が立ちました。しかし、この映画で最も腹が立ったのは、クリストファー・ロイド、ジェフ・ダニエルズ、ダリル・ハンナという才能ある俳優を無駄にしてしまったことです。ジェフ・ダニエルズはこれまでも良い演技をしてきましたが、彼は何をすべきかわからないようでしたし、エリザベス・ハーリーのキャラクターも残念ながら役立たずでした。ダリル・ハンナは素敵な女優だが、一般的には無視されており、私は彼女が愛の対象になるというアイデアが好きだったが、残念ながら彼女の姿はほとんど見られない。（モンスターの攻撃は、子供たちを魅了するというよりも、怖がらせる可能性が高いのは言うまでもない）同様に、ウォレス・ショーンもある種の政府の工作員として登場する。        1/10 ベサニー・コックス

試しに一文をTokenizerで単語IDに変換する

In [ ]:
torch.tensor(tokenizer(raw_train_data['text'][0], max_length=10)['input_ids'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


tensor([  101, 22953,  2213,  4381,  2152,  2003,  1037,  9476,  4038,   102])

必要なパラメタを定義する
- 各データセットの文章数
- バッチサイズ
- 最大の文章長さ(これ以上の長さは切られる)

In [ ]:
num_train_data = raw_train_data.num_rows
num_test_data = raw_test_data.num_rows
batch_size = 32
max_seq_len = 256

mapメソッドを利用して各データに前処理を施す
- ここではtokenizeを定義し、このtokenizeを全データに施す
- tokenizeは読み込んだIMDbのデータをTokenizerで処理し、語句IDに変換する関数である
- バッチサイズはデータ全体、つまり全データに対して一気に処理している(順番に取り出して何かするのではないため、これでよい)
- "input_ids", "attention_mask", "label"の順番にデータを並べて、PyTorchで利用できるようにPyTorchのDataLoaderと同様の形で出力させる
- max_lengthで長い文章をここで制限しておく
  - つけないと512になる

なお、set_formatのtype="torch"は、torch.tensorで出力する指定である
- だが、set_format自体が変換するわけではなく、tokenizerに渡して変換する仕様のようだ
- 従って、指定のtokenizerを利用しなければ変換されないので注意
  - これがAutoTokenizerを使う大きな理由の一つであり、バッチ化を簡単にできる

本来、テストデータはシャッフルする必要はないが、最後に乱雑に確認したいため、シャッフルしている
- **普通ではないので注意**

In [ ]:
train_data = raw_train_data.map(lambda e: tokenizer(e['text'], truncation=True, max_length=max_seq_len, padding='max_length'), batched=True)
train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_data = raw_test_data.map(lambda e: tokenizer(e['text'], truncation=True, max_length=max_seq_len, padding='max_length'), batched=True)
test_data.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True) # ここのシャッフルは意味がないが最後の試行を乱雑にしたい

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

train_dataの中身は次の通り

In [ ]:
train_data

Dataset(features: {'label': ClassLabel(num_classes=2, names=['neg', 'pos'], names_file=None, id=None), 'text': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 25000)

train_dataloaderから試しにデータを取得する

In [ ]:
next(iter(train_loader))

{'label': tensor([0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
         0, 1, 0, 0, 0, 1, 0, 0]),
 'input_ids': tensor([[ 101, 2066, 2070,  ...,    0,    0,    0],
         [ 101, 2023, 2003,  ...,    0,    0,    0],
         [ 101, 5674, 2008,  ..., 2000, 1037,  102],
         ...,
         [ 101, 1045, 1005,  ...,    0,    0,    0],
         [ 101, 2026, 2564,  ..., 1026, 7987,  102],
         [ 101, 2242, 2000,  ..., 7472, 2007,  102]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]])}

語彙数、つまりTokenizerが知っている単語の種類の数をパラ目として設定する

In [ ]:
vocab_size = tokenizer.vocab_size
vocab_size

30522

## Transformerのネットワーク構造


### 各層の結合とデータサイズ

入カはミニバッチ数$M(=256)$、一文の単語数$W(=256)$とすると、$M\times W$

処理は次の通りとなる

- 内部の単語の表現ベクトル$V(=300)$とすると、Embedderにより、単語一つがV次元のベクトル表現になり、その出力は$M\times W \times V$となる
  - Embedding層は内部で、まず入力記号OneHotベクトルに変換し、そのOneHotベクトルをより低次のベクトル空間上に線形写像している
- Embedderの後、PositionalEncoder、TransformerBlockへと処理が映るが、これらは全て入出力で次元を変更していない
- 最後のTransformerBlockの出力がClassificationHeadにおいて、クラス数$C(=2)$に変換され、結果的に$M\times C$となる

### 各層の動作内容

- Embedding
  - ここでは、PyTorchが提供するnn.Embeddingを用いており、誤差逆伝搬により更新される
  - その他、fasttextや、Word2Vecなどによる事前学習に基づいた分散表現変換も想定される
- PositionalEncoder
  - 入カデータに位置情報を足し算する
  - Self-Attentionを利用するため、各単語がどの単語と関係するかはAttentionで計算、獲得できる
  - すると、入力文章の単語の順番がシャッフルされた場合、同様に処理すると、語順という概念が欠落して同じ結果が出る可能性がある、すなわち語順が考慮されない、という問題を解決する
  - 今回のように文章の構造を判断材料に入れたいという場合に導入している

- TransformerBlock
  - 任意の回数繰り返して利用する
    - Transformerの図で$\times n$と記載されている通り
  - ここでは2段構成となっている
  - 入力のmaskはAttention Mapの一部の値を0に置き換える
  - 文章がmax_lengthの256文字よりも短くパディング、つまり<pad>が埋め込まれている部分についてAttentionを求めないように、その重みを0とする
  - 翻訳タスクなどのデコーダ側では、マスクされた単語を補完する、マスク位置を次々とずらすことで文章を完成させるといったタスクを達成するために利用する

- ClassificationHead
  - 今回のタスクがクラス分けであるため、Transformer標準ではないが、最後に設けて次元数2の出力に変換する

### Embedder

既述は次の通り

主なオプションは次の通り
- `num_embeddings（int）`: 埋め込み辞書のサイズ
- `embedding_dim（int）`: 各埋め込みベクトルのサイズ
- `freeze=True`: ここでは利用していないが、誤差逆伝搬において内部重みの更新を阻止する

In [ ]:
class Embedder(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super(Embedder, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim)

    def forward(self, x):
        x_vec = self.embeddings(x)
        return x_vec

Embedderの動作を確認する

In [ ]:
# モデル構築
net1 = Embedder(vocab_size,300)
# 入出力
test_batch = next(iter(train_loader))
x = test_batch['input_ids']
x1 = net1(x)  # 単語をベクトルに
print("入力のテンソルサイズ：", x.shape)
print("出力のテンソルサイズ：", x1.shape)

入力のテンソルサイズ： torch.Size([32, 256])
出力のテンソルサイズ： torch.Size([32, 256, 300])


### PositionalEncoder

入力された単語の位置を示すベクトル情報`pe`を付加する
- 位置の計算式はTransfomerの論文のままの標準的な方法
- 文章が短く、単語ベクトルがPositional Encodingよりも小さい場合に対応するため、$\sqrt{V}$を掛けて大きさをある程度そろえる処理が加わっている
- `pe`は何度も計算するわけではなく、コンストラクタにおいて、テーブルとして保持している
- `pe`は勾配計算の対象外であるため、`requires_grad = False`を忘れないように
  - 計算しても動作するがpeが更新され変更される
  - 結果として実行速度低下を招く
  - 精度低下を招くかどうかはなんともいえない
    - 課題としてトライしてみるとよいだろう

In [ ]:
class PositionalEncoder(nn.Module):
    def __init__(self, d_model=300, max_seq_len=max_seq_len, devname='cpu'):
        super().__init__()
        self.d_model = d_model  # 単語ベクトルの次元数
        pe = torch.zeros(max_seq_len, d_model)
        pe = pe.to(devname)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * i)/d_model)))
        self.pe = pe.unsqueeze(0) # 表peの先頭に、ミニバッチ次元となる次元を足す
        self.pe.requires_grad = False # 勾配を計算しないようにする
    def forward(self, x):
        # 入力xとPositonal Encodingを足し算する
        # xがpeよりも小さいので、大きくする
        ret = math.sqrt(self.d_model)*x + self.pe
        return ret

PositonalEncoderの動作確認を行う

In [ ]:
# モデル構築
net2 = PositionalEncoder(d_model=300, max_seq_len=max_seq_len)
# 入出力
x = test_batch['input_ids']
x1 = net1(x)  # 単語をベクトルに
x2 = net2(x1)
print("入力のテンソルサイズ：", x1.shape)
print("出力のテンソルサイズ：", x2.shape)

入力のテンソルサイズ： torch.Size([32, 256, 300])
出力のテンソルサイズ： torch.Size([32, 256, 300])


### TransformerBlock

LayerNormalization、Dropout、Attention、FeedForwardで構成

- LayerNormalization: 各単語が持つ$V$個の特徴量に対し、その特徴量毎に正規化を行う
  - 各特徴量が持つ$V$次元の要素の平均と標準偏差が、それぞれ0と1になるように正規化

- Attentionにおいて特徴量が変換
- その出力にDropoutしたベクトルとLayerNormalizationの入力のベクトルを足し算する
- FeedForwardにより特徴量変換を行う

なお、オリジナルのTransformerはMulti-Head Attentionであるが、ここではSingle Attentionで実装している
- Single Attentionを複数並列するとMulti-Headになる
- Milti-headについては演習で扱う

- テキストの隙間埋めパディング`<pad>`の部分のmask値は0であるが、Attentionにおいてはこの部分を-le9というマイナス無限大に近い値に置き換える
- 結果的に、その後のSofmax計算で邪魔をしなくなる
  - Attention Mapにおいて0になるようにするため

<img src="https://class.west.sd.keio.ac.jp/dataai/text/mytransformerblock.png" width=500>

In [ ]:
class Attention(nn.Module):
    def __init__(self, d_model=300):
        super().__init__()
        # SAGANでは1dConvを使用したが、今回は全結合層で特徴量を変換する
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.out = nn.Linear(d_model, d_model)  # 出力時に使用する全結合層
        self.d_k = d_model  # Attentionの大きさ調整の変数

    def forward(self, q, k, v, mask):
        # 全結合層で特徴量を変換
        k = self.k_linear(k)
        q = self.q_linear(q)
        v = self.v_linear(v)
        # Attentionの値を計算する
        weights = torch.matmul(q, k.transpose(1, 2)) / math.sqrt(self.d_k)  # 値が大きくならないようroot(d_k)で割って調整
        mask = mask.unsqueeze(1) # maskを計算
        weights = weights.masked_fill(mask == 0, -1e9)
        normlized_weights = F.softmax(weights, dim=-1)  # softmaxで正規化
        output = torch.matmul(normlized_weights, v)  # AttentionをValueとかけ算
        output = self.out(output)  # 全結合層で特徴量を変換
        return output, normlized_weights

Attention層から出力を全結合層2つで特徴量を変換する

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=1024, dropout=0.1):
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        x = self.linear_1(x)
        x = self.dropout(F.relu(x))
        x = self.linear_2(x)
        return x

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model, dropout=0.1):
        super().__init__()
        # LayerNormalization層
        self.norm_1 = nn.LayerNorm(d_model)
        self.norm_2 = nn.LayerNorm(d_model)
        # Attention層
        self.attn = Attention(d_model)
        # Attentionのあとの全結合層
        self.ff = FeedForward(d_model)
        # Dropout
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)

    def forward(self, x, mask):
        x_normlized = self.norm_1(x)  # 正規化
        output, normlized_weights = self.attn(  # Attention
            x_normlized, x_normlized, x_normlized, mask)
        x2 = x + self.dropout_1(output)
        x_normlized2 = self.norm_2(x2)  # 正規化
        output = x2 + self.dropout_2(self.ff(x_normlized2)) # 全結合層
        return output, normlized_weights

## 動作確認

In [ ]:
net3 = TransformerBlock(d_model=300)  # モデル構築
# maskの作成
x = test_batch['input_ids']
input_pad = 101  # 単語のIDにおいて、'<pad>': 1 なので
input_mask = (x != input_pad)
# 入出力
x1 = net1(x)  # 単語をベクトルに
x2 = net2(x1)  # Positon情報を足し算
x3, normlized_weights = net3(x2, input_mask)  # Self-Attentionで特徴量を変換
print("入力のテンソルサイズ：", x2.shape)
print("出力のテンソルサイズ：", x3.shape)
print("Attentionのサイズ：", normlized_weights.shape)

入力のテンソルサイズ： torch.Size([32, 256, 300])
出力のテンソルサイズ： torch.Size([32, 256, 300])
Attentionのサイズ： torch.Size([32, 256, 256])


### ClassificationHead

評価のPositive/Negativeの2つのクラス分類を出力する

ここで、どこを利用して特徴量を抽出するかについて選択肢がいくつかある
- 文章全体を用いて特徴量を抽出する
  - もちろん可能であり、こちらの方がよさそうですが、問題として文章の長さが異なるためパディングによる悪影響が回避できるかどうかが疑わしいという問題がある
  - これは、課題として比較してみるとよいであろう
- どこかしらの1つの特徴量を利用する
  - こうなるとどこか、ということであるが、先頭単語の特徴量を利用するという方針を選択している
  - これは、先頭単語に分類に必要な特徴量が存在するというわけではない
  - 学習によって、「そうなるように」能力を獲得させるということである
  - これでもうまくいくのだから、DNNはそれなりにミスがあっても、見当違いがあっても、おおらかに、かつ甘んじてそれを受け入れ、その制約の中で頑張って学習する健気な存在である

In [ ]:
class ClassificationHead(nn.Module):
    '''Transformer_Blockの出力を使用し、最後にクラス分類させる'''

    def __init__(self, d_model=300, output_dim=2):
        super().__init__()

        # 全結合層
        self.linear = nn.Linear(d_model, output_dim)  # output_dimはポジ・ネガの2つ

        # 重み初期化処理
        nn.init.normal_(self.linear.weight, std=0.02)
        nn.init.normal_(self.linear.bias, 0)

    def forward(self, x):
        x0 = x[:, 0, :]  # 各ミニバッチの各文の先頭の単語の特徴量（300次元）を取り出す
        out = self.linear(x0)

        return out


Transformerの動作確認

In [ ]:
batch = next(iter(train_loader))  # ミニバッチの用意
# モデル構築
net3 = TransformerBlock(d_model=300)
net4 = ClassificationHead(output_dim=2, d_model=300)
# 入出力
x =test_batch['input_ids'][0]
x1 = net1(x)  # 単語をベクトルに
x2 = net2(x1)  # Positon情報を足し算
x3, normlized_weights = net3(x2, input_mask)  # Self-Attentionで特徴量を変換
x4 = net4(x3)  # 最終出力の0単語目を使用して、分類0-1のスカラーを出力
print("入力のテンソルサイズ：", x3.shape)
print("出力のテンソルサイズ：", x4.shape)

入力のテンソルサイズ： torch.Size([32, 256, 300])
出力のテンソルサイズ： torch.Size([32, 2])


最終的なTransformerモデルのクラス

In [ ]:
class TransformerClassification(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, d_model=300, max_seq_len=max_seq_len, output_dim=2):
        super().__init__()
        self.net1 = Embedder(num_embeddings, embedding_dim)
        self.net2 = PositionalEncoder(d_model=d_model, max_seq_len=max_seq_len, devname=device)
        self.net3_1 = TransformerBlock(d_model=d_model)
        self.net3_2 = TransformerBlock(d_model=d_model)
        self.net4 = ClassificationHead(output_dim=output_dim, d_model=d_model)

    def forward(self, x, mask):
        x1 = self.net1(x)  # 単語をベクトルに
        x2 = self.net2(x1)  # Positon情報を足し算
        x3_1, normlized_weights_1 = self.net3_1(
            x2, mask)  # Self-Attentionで特徴量を変換
        x3_2, normlized_weights_2 = self.net3_2(
            x3_1, mask)  # Self-Attentionで特徴量を変換
        x4 = self.net4(x3_2)  # 最終出力の0単語目を使用して、分類0-1のスカラーを出力
        return x4, normlized_weights_1, normlized_weights_2

最終的なTransformerモデルのクラスの動作確認

In [ ]:
model = TransformerClassification(  # モデル構築(batchは前の値を利用する)

    num_embeddings=vocab_size, embedding_dim=300, d_model=300, max_seq_len=max_seq_len, output_dim=2).to(device)

# 入出力
x = test_batch['input_ids']
x = x.to(device)
input_mask = (x != input_pad).to(device)
out, normlized_weights_1, normlized_weights_2 = model(x, input_mask)

print("出力のテンソルサイズ：", out.shape)
print("出力テンソルのsigmoid：", F.softmax(out, dim=1))


出力のテンソルサイズ： torch.Size([32, 2])
出力テンソルのsigmoid： tensor([[1.0000e+00, 1.3675e-06],
        [1.0000e+00, 1.7600e-06],
        [1.0000e+00, 1.7467e-06],
        [1.0000e+00, 1.5209e-06],
        [1.0000e+00, 1.4390e-06],
        [1.0000e+00, 1.3932e-06],
        [1.0000e+00, 1.5905e-06],
        [1.0000e+00, 1.8700e-06],
        [1.0000e+00, 1.7951e-06],
        [1.0000e+00, 1.7382e-06],
        [1.0000e+00, 1.7375e-06],
        [1.0000e+00, 1.7609e-06],
        [1.0000e+00, 1.5063e-06],
        [1.0000e+00, 1.8945e-06],
        [1.0000e+00, 1.7396e-06],
        [1.0000e+00, 1.3070e-06],
        [1.0000e+00, 1.4250e-06],
        [1.0000e+00, 1.7798e-06],
        [1.0000e+00, 1.4900e-06],
        [1.0000e+00, 1.5721e-06],
        [1.0000e+00, 1.7312e-06],
        [1.0000e+00, 1.7077e-06],
        [1.0000e+00, 1.7074e-06],
        [1.0000e+00, 1.6516e-06],
        [1.0000e+00, 1.7173e-06],
        [1.0000e+00, 1.6300e-06],
        [1.0000e+00, 1.5699e-06],
        [1.0000e+00, 1.5289e-06],


### DatasetとDataLoaderの実装

In [ ]:
# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_loader, "val": test_loader}

ネットワークの初期化として、 He の方法 (一様分布)を用いる
- 初期化については後で概要についてまとめる

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        # Liner層の初期化
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)
# 訓練モードに設定
model.train()
# TransformerBlockモジュールを初期化実行
model.net3_1.apply(weights_init)
model.net3_2.apply(weights_init)
model.to(device)  # モデルをGPUへ

TransformerClassification(
  (net1): Embedder(
    (embeddings): Embedding(30522, 300)
  )
  (net2): PositionalEncoder()
  (net3_1): TransformerBlock(
    (norm_1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
    (norm_2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
    (attn): Attention(
      (q_linear): Linear(in_features=300, out_features=300, bias=True)
      (v_linear): Linear(in_features=300, out_features=300, bias=True)
      (k_linear): Linear(in_features=300, out_features=300, bias=True)
      (out): Linear(in_features=300, out_features=300, bias=True)
    )
    (ff): FeedForward(
      (linear_1): Linear(in_features=300, out_features=1024, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear_2): Linear(in_features=1024, out_features=300, bias=True)
    )
    (dropout_1): Dropout(p=0.1, inplace=False)
    (dropout_2): Dropout(p=0.1, inplace=False)
  )
  (net3_2): TransformerBlock(
    (norm_1): LayerNorm((300,), eps=1e-05, elementwise_af

### 損失関数と最適化手法を定義

In [ ]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

# 最適化手法の設定
learning_rate = 2e-5
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### 学習・検証

モデルを学習させる関数を作成

In [ ]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    torch.backends.cudnn.benchmark = True  # ネットワークがある程度固定であれば高速化る
    for epoch in range(num_epochs):  # epochのループ
        for phase in ['train', 'val']:  # epochごとの訓練と検証のループ
            if phase == 'train':
                model.train()  # モデルを訓練モードに
            else:
                model.eval()  # モデルを検証モードに
            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            for batch in (dataloaders_dict[phase]):  # データローダーからミニバッチを取り出す
                inputs = batch['input_ids'].to(device)  # 文章を可能ならばGPUへ
                labels = batch['label'].to(device)  # ラベルを可能ならばGPUへ
                optimizer.zero_grad()  # optimizerを初期化
                with torch.set_grad_enabled(phase == 'train'):  # 順伝搬の計算
                    # mask作成
                    input_mask = (inputs != input_pad)
                    input_mask = input_mask.to(device)
                    outputs, _, _ = model(inputs, input_mask)  # Transformerに入力
                    loss = criterion(outputs, labels)  # 損失を計算
                    _, preds = torch.max(outputs, 1)  # ラベルを予測
                    if phase == 'train':  # 訓練時のみ勾配計算と更新
                        loss.backward()
                        optimizer.step()
                    epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)  # 正解数の合計を更新
            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)
            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                phase, epoch_loss, epoch_acc))
    return net

In [ ]:
# 学習・検証を実行する 15分ほどかかります
num_epochs = 10
net_trained = train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

Epoch 1/10 | train |  Loss: 0.7789 Acc: 0.5697
Epoch 1/10 |  val  |  Loss: 0.6304 Acc: 0.6430
Epoch 2/10 | train |  Loss: 0.6217 Acc: 0.6538
Epoch 2/10 |  val  |  Loss: 0.5998 Acc: 0.6763
Epoch 3/10 | train |  Loss: 0.5895 Acc: 0.6848
Epoch 3/10 |  val  |  Loss: 0.5795 Acc: 0.6913
Epoch 4/10 | train |  Loss: 0.5417 Acc: 0.7272
Epoch 4/10 |  val  |  Loss: 0.5277 Acc: 0.7352
Epoch 5/10 | train |  Loss: 0.5042 Acc: 0.7528
Epoch 5/10 |  val  |  Loss: 0.5229 Acc: 0.7376
Epoch 6/10 | train |  Loss: 0.4827 Acc: 0.7654
Epoch 6/10 |  val  |  Loss: 0.4888 Acc: 0.7639
Epoch 7/10 | train |  Loss: 0.4645 Acc: 0.7780
Epoch 7/10 |  val  |  Loss: 0.4879 Acc: 0.7635
Epoch 8/10 | train |  Loss: 0.4506 Acc: 0.7881
Epoch 8/10 |  val  |  Loss: 0.4739 Acc: 0.7726
Epoch 9/10 | train |  Loss: 0.4390 Acc: 0.7957
Epoch 9/10 |  val  |  Loss: 0.4681 Acc: 0.7755
Epoch 10/10 | train |  Loss: 0.4223 Acc: 0.8040
Epoch 10/10 |  val  |  Loss: 0.4569 Acc: 0.7845


正解率の計算と表示

In [ ]:
net_trained.eval()   # モデルを検証モードに
net_trained.to(device)
epoch_corrects = 0  # epochの正解数
for batch in (test_loader):  # testデータのDataLoader
    inputs = batch['input_ids'].to(device)
    labels = batch['label'].to(device)
    with torch.set_grad_enabled(False):  # 順伝搬のみ計算
        input_mask = (inputs != input_pad)  # mask作成
        outputs, _, _ = net_trained(inputs, input_mask)  # Transformerに入力
        _, preds = torch.max(outputs, 1)  # ラベルを予測
        epoch_corrects += torch.sum(preds == labels.data)  # 結果の計算

正解率の出力

In [ ]:

epoch_acc = epoch_corrects.double() / len(test_loader.dataset)
print('テストデータ{}個での正解率：{:.4f}'.format(len(test_loader.dataset),epoch_acc))

テストデータ25000個での正解率：0.7845


## Attentionの可視化と判定根拠の判断



### HTML作成関数の実装

なぜレピュー文章の内容をポジテイプもしくはネガティブとモデルが判定したのか、判定する際に強くAttentionをかけた単語を可視化することで、その判定根拠を探る
- XAI(Explainable Artificial Intelligence :説明可能AI)の議論への対応として、説明性を持たせる判定根拠の可視化は重要
- 自然言語処理における判定根拠を明確に示す手法は確立していない
- Attentionが判定根拠になるかどうかは議論となっている

文章の各単語についてAttentionの影響が強い単語の背景(HTMLのbackground-colorスタイル)ほどより赤くハイライトする
- Jupyter NotebookはHTML表示に対応するため、HTMLデータとして作成して表示する
- 文章の1単語目に埋め込まれているである<cls>の特徴量が分類の判断材料であるため、この特徴量を作成するために利用したSelf-Attentionをnormlized_weights から取り出して仕様する
  - TransformerBlockモジュールが2つあるため、1つ目と2つ目のttention
が存在する

関数は次の2つ
- highlight
  Attentionの値が大きいと文字の背景が濃い赤になるhtmlを出力させる関数
- mk_html
  実際にHTMLデータを作成する


In [ ]:
def highlight(word, attn):
    if (word == input_pad or word == 0):
        return ''
    wordc = tokenizer.convert_ids_to_tokens([word])[0]
    html_color = '#%02X%02X%02X' % (
        255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, wordc)

def mk_html(index, batch, preds, normlized_weights_1, normlized_weights_2):
    # indexの結果を抽出
    sentence = batch['input_ids'][index]  # 文章
    label = batch['label'][index]  # ラベル
    pred = preds[index]  # 予測
    # indexのAttentionを抽出と規格化
    attens1 = normlized_weights_1[index, 0, :]  # 0番目の<cls>のAttention
    attens1 /= attens1.max()
    attens2 = normlized_weights_2[index, 0, :]  # 0番目の<cls>のAttention
    attens2 /= attens2.max()
    # ラベルと予測結果を文字に置き換え
    if label == 0:
        label_str = "Negative"
    else:
        label_str = "Positive"
    if pred == 0:
        pred_str = "Negative"
    else:
        pred_str = "Positive"
    # 表示用のHTMLを作成する
    html = '正解ラベル：{}<br>推論ラベル：{}<br><br>'.format(label_str, pred_str)
    # 1段目のAttention
    html += '[TransformerBlockの1段目のAttentionを可視化]<br>'
    for word, attn in zip(sentence, attens1):
        html += highlight(word, attn)
    html += "<br><br>"
    # 2段目のAttention
    html += '[TransformerBlockの2段目のAttentionを可視化]<br>'
    for word, attn in zip(sentence, attens2):
        html += highlight(word, attn)
    html += "<br><br>"
    return html

実行するたびに異なる文章を評価できる

- Positiveな文章ではPositiveな単語が、Negativeでは逆の単語に注目していることがわかる
- 結果を見てどのような文章で誤解しているのかなどを解析し、さらなる工夫を施すことが考えられる

In [ ]:
from IPython.display import HTML

# Transformerで処理

# ミニバッチの用意
batch = next(iter(test_loader))

# GPUが使えるならGPUにデータを送る
inputs = batch['input_ids'].to(device)  # 文章
labels = batch['label'].to(device)  # ラベル

# mask作成
input_mask = (inputs != input_pad)

# Transformerに入力
outputs, normlized_weights_1, normlized_weights_2 = net_trained(
    inputs, input_mask)
_, preds = torch.max(outputs, 1)  # ラベルを予測


#index = 3  # 出力させたいデータ
index = random.randint(0, batch_size-1)
html_output = mk_html(index, batch, preds, normlized_weights_1,
                      normlized_weights_2)  # HTML作成
HTML(html_output)  # HTML形式で出力


# 課題(Transformer)

- 説明文中で言及した次の2つの課題について実際に試しなさい
  - Embeddingの値を学習させた場合とさせない場合の結果の違い
  - ClassificationHeadにおける特徴量の扱い方における結果の違い
    - この場合ハイライティングは言及しなくてよい

- 日本語で実験してみよう
  - https://github.com/amazon-research/amazon-multilingual-counterfactual-dataset
  - こちらのデータセット利用してトライする
    - ただ、中身を見るとわかるが、ちょっとつまらないかも

- Embeddingによる内部の単語表現ベクトルの次元を変えたとき、精度にどのように影響するかを調査しなさい
  - できれば減らせるようにしよう

- Transformerの段数を増やし、精度が向上するか確認してみよう

- LightGBMと精度を比較してみなさい
  - 落胆する結果にならないとよいですが…

# Transformer
- 2017年に導入されたディープラーニングモデルの一種
  - 主に自然言語処理で利用されている
- RNNと同様自然言語などの時系列データ処理向けに設計されているが、再帰や畳み込みは利用していない
- Attention層のみで構築されている(後述)
- 翻訳やテキスト要約などの各種タスクに利用可能
- 並列化が容易で訓練時間を削減できる
- 「Attention is All You Need」という論文で著名になった
- 機械翻訳タスクにおいてRNNを用いたモデルよりも精度がよく、訓練コストが小さいことから革命的であり、NLPではRNNに印籠を渡し現在の主流の方法である
  - この後登場するBERT、ELECTRA、GPTなどすべてTransformerを基本としている

RNNとは全く異なるアプローチ
- EncoderおよびDecoderのいずれにもRNNのような再帰構造をもたず、Attentionが利用されている
- その優れた特徴から、自然言語処理以外の分野でも利用が進む
- PyTorchでは既に公式実装が存在しておりそちらを利用するべきであるが、ここでは構造を理解するため全て記述する

### Transformerの構造

<img src="http://class.west.sd.keio.ac.jp/dataai/text/transformer.png" width=500>

Seq2Seq同様EncoderとDecoderで構成

### Encoderの構造
1. Embedding層により入力文章をベクトルに圧縮、つまり分散表現に変換する
1. Positional Encoder層で文章内のどこにあるかという位置情報を加える
1. Multi-Head Attention層(後述)
1. normalization(正規化)によりデータの偏りを削減する
  - batch normalizationではなくlayer normalizationが行なわれる
1. Feed Forward層と組み合わせて処理され、実際のモデルでは例えば6回繰り返される
  - 出力されたベクトルはDecoderに渡される
  - 特にPositionwise fully connected feed-forward networkと呼ばれる

- Multi-Head Attention層とFeed Forward層の組み合わせが6回繰り返される

以上で、Encoderが構成される

### Decoderの構造

1. Embedding層により入力文章をベクトルに圧縮(分散表現を獲得)
1. Positional Encoder層で位置情報を追加
1. Masked Multi-Head Attention層、先ほどと同様であるがAttention内のsoftmax関数を通す直前の値にマスキングが適用されている
  - 特定のkeyに対して、Attention weightを0にすることで入力した単語の先読みによる「カンニング」を防ぐ
  - 入力に予測すべき結果が入らないようにする
1. normalization（正規化）などで先ほどと同様
1. Multi-Head Attention層（Encoderの出力を入力として使用）
1. normalization（正規化）など
1. Positionwise fully connected feed-forward network(先ほどと同じ)
1. normalization（正規化）など
- 例えば、以上を6回繰り返す



### Transformerの構成要素

- Attention
  -「文章中のどの単語に注目すればよいかを表すスコア」のこと
  - Query、Key、Valueの3つのベクトルで求める
    - Query: Inputのうち「検索をかけたいもの」
    - Key: 検索対象とQueryの近さ、どれだけ似ているかを測る
    - Value: Keyに基づき、適切なValueを出力する
  - Self-Attention
    - 下図でInputとMemoryが同一のAttention
      - 文法の構造や、単語同士の関係性などを獲得するのに使用される
  - SourceTarget-Attention
    - 下図でInputとMemoryが異なるAttention
      - TransformerではDecoderで使用される
  - Multi-Head Attention
    - Attentionを複数並列して並べたもの(後述)
  - Masked Multi-Head Attention
    - Multi-Head Attentionにマスクをつけたもの
    - 特定の key に対してAttention weight を0にする
    - TransformerではDecoderで使われる
    - 入力した単語が先読みを防ぐために 情報をマスクで遮断する、言わば「カンニング」を防ぐ
  - Attentionは可視化できる
    - すでに示したが、attentionは可視化でき、どの単語に注目しているかを知ることができる
- Position-wise Fully-connected Feedforward Network
  - 2層からなる全結合ニューラルネットワーク
  - 単語の位置ごとに個別の順伝播ネットワークとなる
    - これにより他単語との影響関係を排除することができる
  - パラメータは全てのネットワークで共通
$FNN(x) = LeRU(xW_1+b_1)\cdot W_2+b_2$
- Positional Encoding ($PE$)
  - 「単語の位置」の情報をベクトルに加える
    - 本当に加えるだけで次元を増やさない
  - $pos$は位置を表し、$2i$および$2i+1$はEmbeddingの何番目の次元か、$d_{model}$が次元数を示す
偶数番目：$PE_{(pos,2i)}=sin(pos/10000^{2i/d_{model}})$
奇数番目：$ PE_{(pos,2i+1)}=cos(pos/10000^{2i/d_{model}})$

<img src="http://class.west.sd.keio.ac.jp/dataai/text/attention2.png" width=800>

- 丸角(緑)がベクトル(テンソル)、四角角(青)が処理を表す
- **InputとMemoryはそれぞれ異なる埋め込みベクトルを表し、例えば2つの異なる文章を表す**
- Inputについて全結合層で各単語のQueryを作成する
- Memoryについても同様に全結合層でKeyを作成しQueryとの内積をとって関連度合い見る
  - 同じ向きを向いていれば掛け算となる
  - 垂直である、つまり関連しなければ0
  - この値を関連度(logit)とする
- logitにSoftmaxを適用して0から1の間に調整して出力、この結果が Attention weightとなる
  - メモリのどの単語に注意を払うかの重みづけ
  - QueryとKeyの関連が大きいとAttention weightが大きくなる
    - 正しくMemoryの単語に注意を向けるように,keyが正しくAttentionに向けられるように学習される
- Memoryから全結合層を経て、Memoryの各単語に対する埋め込みベクトルであるValueを算出する
  - ValueとAttenthion weightとの内積を求める
    - Attention weightに従ってValueを選択することを意味する
- 最後に全結合層を挟んで出力を得る




### InputとMemory

<img src="http://class.west.sd.keio.ac.jp/dataai/text/input-memory.png" width=600>

各文章は分かち書きされIDで表現された後、Embeddingにより埋め込みベクトルに変換される

### Attention Weightの算出

<img src="http://class.west.sd.keio.ac.jp/dataai/text/attention-weight.png" width=600>

QueryとKeyの内積を算出してInputとMemoryの各単語の関連度であるlogitを算出、Softmaxを用いてAttention weightとする
- Memoryのどの単語に注意を払うかの重み付け

例えば、Inputのスポーツという単語に対して、Memoryの「野球」 「が」 「得意」の各単語について正しく注意を向けるように学習する
- ここでは野球が高い値になるようになる

### valueとの内積

<img src="http://class.west.sd.keio.ac.jp/dataai/text/value-naiseki.png" width=600>

この内積は、value、ここでは「野球」「が」「得意」の各単語のValueとAttention weightを掛け合わせて総和を計算することになる

最も注目するべきvalueの値が算出されているといえるが、他の単語との関連性も考慮した値となっている

### Multi-Head Attention

<img src="http://class.west.sd.keio.ac.jp/dataai/text/mhattention.jpg" width=200>

- Attentionを並列させた構造を持つ
- それぞれのAttentionをHeadと呼ぶ
- 「Attention Is All You Need」ではMulti-Head化により性能が向上するとされている
  - アンサンブル学習に近い
  - Dropoutも毎回異なるネットワークを使っており、アンサンブルに通じるところがある

## Positional Encoding

Postional Encoding層は、系列データ内の各要素に、要素のデータ内における位置情報を付与する
- 文章の場合、Positiona Encodingによって、各単語ベクトルに、その単語が文章内で何番目に登場するかという情報を付与する

次の式で算出した値を固定値として、入力に加算する

$PE_{(pos,2i)} = sin(pos/10000^{2i/d_{model}})$

$PE_{(pos,2i+1)} = cos(pos/10000^{2i/d_{model}})$

このPositional Encodingの値を図示すると次のようになる

<img src="http://class.west.sd.keio.ac.jp/dataai/text/pos_encoding.png" width=400>


# PyTorch Transformerを用いた単語予測

Transformerは複雑な構造をもっており、これそのものをPytorchで記述することも可能であるが、CNNやRNNと同様、PyTorchが提供するライブラリを利用することで簡単に利用できる
- ここではPytorch Transformerが提供するTransformerを構築するに必要な層の要素を組み合わせて設計する

Pytorh TransformerとTorchTextを用い、先に学んだsequence-to-sequenceモデルを使って機械翻訳モデルを実装する
- この内容はPyTorchのチュートリアルドキュメントに準拠する

WikiText2から取得した文章を用いて単語系列であるsequenceを入力、次に来る単語の予測を行う


## モデル定義

単語、つまりトークンの並びであるシーケンスがモデルに入力されると、位置エンコーディング層で単語の順序情報が加えられる

言語モデルタスクでは、入力シーケンスと共に、アテンション・マスクを利用する
- nn.TransformerEncoderのSelf-Attention層では、シーケンスにおけるその単語以前の単語のみ知ることができる
  - 普通は、未来に登場するはずの単語は考慮できないため
- そこで、言語モデルタスクでは、後で登場するトークンは未知のトークンとして扱う必要があるため、これらをマスクする


nn.TransfomerEncoderについて
- nn.TransformerEncoderモデルの出力は、最終的に全結合層に送られlog-Softmax関数を介することで予測結果を得ることができる
- nn.TransformerEncoderは、複数のnn.TransformerEncoderLayer層で構成されており並列的に動作できるためRNNよりも計算効率が良い

TransformerModelの引き数
- src: [seq_len, batch_size]のTensor型
- src_mask: [seq_len, seq_len]のTensor型
戻り値: [seq_len, batch_size, ntoken]のTensor型


In [ ]:
import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

class TransformerModel(nn.Module):
    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

def generate_square_subsequent_mask(sz: int) -> Tensor:
    # -infの上三角行列を生成し，対角線上に0を置く。
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

## PositionalEncoding

PosigionalEncodingモジュールは、シーケンス内のトークンの相対的な位置、もしくは絶対的な位置に関する情報を与える

オリジナルの実装と同様、入力と出力は同じ次元である
- つまり、もともとの入力$x$に対して、PositionalEncodingの値$p(x)$が得られたとすると、出力は$x+p(x)$となる

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        # [seq_len, batch_size, embedding_dim]型Tensorを引数とする
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

## データの読み込みとバッチ処理


ここでは、torchtextのWikitext-2データセットを利用する

vocabはトークン（単語）をテンソル形式の数値に変換する
- 訓練データセットを元に構築されており、データセット依存である

- batchify()関数は、トークンが左から右に一つずつ並んだシーケンス形式のデータを束ねて、batch処理ができるようにする
  - 変換にはデータをbatch_size 変数のサイズで分割し、余ったトークンは廃棄する

例えば、アルファベット26文字をシーケンスとしたとき、バッチサイズが4であれば、アルファベットを長さ6の4つのシーケンスに分割することが考えられる

この時次のような変換が行なわれる

\begin{align}\begin{bmatrix}
  \text{A} & \text{B} & \text{C} & \ldots & \text{X} & \text{Y} & \text{Z}
  \end{bmatrix}
  \Rightarrow
  \begin{bmatrix}
  \begin{bmatrix}\text{A} \\ \text{B} \\ \text{C} \\ \text{D} \\ \text{E} \\ \text{F}\end{bmatrix} &
  \begin{bmatrix}\text{G} \\ \text{H} \\ \text{I} \\ \text{J} \\ \text{K} \\ \text{L}\end{bmatrix} &
  \begin{bmatrix}\text{M} \\ \text{N} \\ \text{O} \\ \text{P} \\ \text{Q} \\ \text{R}\end{bmatrix} &
  \begin{bmatrix}\text{S} \\ \text{T} \\ \text{U} \\ \text{V} \\ \text{W} \\ \text{X}\end{bmatrix}
  \end{bmatrix}\end{align}

なお、各バッチ、つまり各列はモデル内ではそれぞれ独立しており、その境界を越えて依存関係を学習することはできない
- 例えばFとGの依存関係を学習することはできない
- それでも大量にデータを入力して学習させるため問題とはならない
- バッチ処理を有効に活用した方が計算効率が高く、その方がメリットが大きい


Google Colaboratoryにはtorchdataがないので、インストール

ランタイムを再起動させる必要があるかもしれないので注意

In [ ]:
!pip install torchdata

必要なライブラリの読み込み



In [ ]:
!pip install torchtext

`portalocker.Lock`でエラーが発生するため、torchtext.datasetsを読み込む前にportalockerをインストールする必要がある
- こういうノウハウ的なところはひとまず気にしなくてよい


In [ ]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

`data_process`は、テキストをテンソル列に変換する

`batchfy`は、データをbsz個のシーケンスに分割し，きれいに収まらない余分な要素を削除する
- data: Tensor, shape [N]
- bsz: int, batch size
- 戻り値: Tensor of shape [N // bsz, bsz]

**<font color="red">+++注意+++</font>**

次のエラーが出力される場合、ランタイムを再起動して対処してください。

最初からやり直すと時間がかかるため、「PyTorch Transformerを用いた単語予測」から以降をランタイムを再起動(Ctrl+M .[ピリオド])して、再実行(Ctrl+F10)するとよい


```
AttributeError: 'NoneType' object has no attribute 'Lock'
This exception is thrown by __iter__ of _MemoryCellIterDataPipe(remember_elements=1000, source_datapipe=_ChildDataPipe)
```



In [ ]:
!pip install portalocker

In [ ]:
train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

# train_iter was "consumed" by the process of building the vocab,
# so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batchify(data: Tensor, bsz: int) -> Tensor:
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

## 入力シーケンスとTargetシーケンスの生成

``get_batch()``関数を用いてTransformerモデルの入力シーケンスと、Targetシーケンスを生成する
- ソースデータを変数``bptt``の長さのチャンクデータに細分化する
- 今回の言語モデルタスクでは、入力シーケンスの続きとなる単語をTargetとして学習させる

例えば``bptt``が2の場合は後続する2 つの要素を取得する

- ``get_batch()``関数の返り値``data``の0次元目がチャンクの長さでを表し、Transformerモデルの次元Sと一致する
- dataの1次元目はバッチサイズを示す次元数Nである

- 引数
  - source: [full_seq_len, batch_size]のテンソル
  - i: 整数
- 返り値
  - (data, target)
  - [seq_len, batch_size]のテンソルであるdataと、[seq_len * batch_size]のテンソルであるtarget

In [ ]:
bptt = 35
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

インスタンスの初期化
--------------------

モデルは、以下のハイパーパラメータを使用して設定されます。vocabのサイズは、vocabオブジェクトの長さと同じです。

In [ ]:
ntokens = len(vocab)  # 語彙サイズ
emsize = 200  # 埋め込み次元数
d_hid = 200  # forwardネットワークモデルのnn.TransformerEncoderの次元
nlayers = 2  # nn.TransformerEncoderのnn.TransformerEncoderLayerの数
nhead = 2  # nn.MultiheadAttentionにおけるheadの数
dropout = 0.2  # dropoutの割合
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

語彙数を確認する

In [ ]:
ntokens

28782

モデルの実行
-------------




- CrossEntropyLossとSDGを用いる
- 初期の学習率は5.0としている
- エポック単位で学習率を調整する
  - StepLRを使用して学習率を調整する
  - 訓練中は、勾配爆発を防ぐためnn.utils.clip_grad_normを用いて学習率を調整する

In [ ]:
import copy
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        seq_len = data.size(0)
        if seq_len != bptt:  # only on last batch
            src_mask = src_mask[:seq_len, :seq_len]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            seq_len = data.size(0)
            if seq_len != bptt:
                src_mask = src_mask[:seq_len, :seq_len]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += seq_len * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

エポックを繰り返す

- 検証データの損失がそれまでの実行のなかで最も良い(低い)場合は当該モデルを保存する
- 各エポックの後に学習率を調整する(小さくする)

In [ ]:
best_val_loss = float('inf')
epochs = 10
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss = evaluate(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)

    scheduler.step()

| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 19.07 | loss  8.11 | ppl  3335.74
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch 15.81 | loss  6.88 | ppl   972.47
| epoch   1 |   600/ 2928 batches | lr 5.00 | ms/batch 20.63 | loss  6.44 | ppl   626.90
| epoch   1 |   800/ 2928 batches | lr 5.00 | ms/batch 18.04 | loss  6.30 | ppl   544.71
| epoch   1 |  1000/ 2928 batches | lr 5.00 | ms/batch 15.14 | loss  6.19 | ppl   485.81
| epoch   1 |  1200/ 2928 batches | lr 5.00 | ms/batch 16.92 | loss  6.16 | ppl   473.52
| epoch   1 |  1400/ 2928 batches | lr 5.00 | ms/batch 23.76 | loss  6.11 | ppl   451.98
| epoch   1 |  1600/ 2928 batches | lr 5.00 | ms/batch 18.03 | loss  6.11 | ppl   449.36
| epoch   1 |  1800/ 2928 batches | lr 5.00 | ms/batch 15.56 | loss  6.03 | ppl   414.02
| epoch   1 |  2000/ 2928 batches | lr 5.00 | ms/batch 14.10 | loss  6.02 | ppl   410.74
| epoch   1 |  2200/ 2928 batches | lr 5.00 | ms/batch 14.88 | loss  5.89 | ppl   362.14
| epoch   1 |  2400/ 

テストデータセットでモデルを評価する
-------------------------------------

結果を確認するために、ベストモデルでテスト用データセットを評価する


In [ ]:
test_loss = evaluate(best_model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

| End of training | test loss  5.40 | test ppl   221.88


In [ ]:
evaltext = "I have enjoyed learning about machine learning so much that I want to continue learning even after the class is over."
evaltext = tokenizer(evaltext)
evaltext

['i',
 'have',
 'enjoyed',
 'learning',
 'about',
 'machine',
 'learning',
 'so',
 'much',
 'that',
 'i',
 'want',
 'to',
 'continue',
 'learning',
 'even',
 'after',
 'the',
 'class',
 'is',
 'over',
 '.']

In [ ]:
evalvcab = vocab(evaltext)
evallen = len(evalvcab)
evalvcab

[64,
 49,
 2267,
 2849,
 73,
 1016,
 2849,
 137,
 210,
 16,
 64,
 1315,
 7,
 1259,
 2849,
 249,
 45,
 1,
 526,
 23,
 65,
 3]

In [ ]:
evalmask = torch.zeros(evallen)
evalmask = generate_square_subsequent_mask(len(evalvcab)).to(device)

In [ ]:
model.eval()
evalout = model(torch.tensor(evalvcab).to(device), evalmask.to(device))

In [ ]:
evalcode = list()
for i in range(evallen):
  evalcode.append(torch.argmax(evalout[i], dim=1)[i])

In [ ]:
import numpy as np
evalcode = list()
#np.argsort(evalout[0][0])
for i in range(evallen):
  evalline = evalout[i][i].to('cpu').detach().numpy().copy()
  evalsort = np.argsort(-evalline)
  evalcode.append(evalsort)
evalcode

[array([  11, 1250,  548, ..., 4030, 3103, 1965]),
 array([  51,    8,  152, ..., 5881, 9602, 3080]),
 array([  19,    1,    6, ..., 2814, 2199, 5244]),
 array([   7,    4,   16, ..., 6678, 2725, 6764]),
 array([  27,   46,    1, ..., 1165, 3872, 5244]),
 array([ 550,  651,    3, ..., 2580, 1965, 2904]),
 array([   7,    4,   16, ..., 6678, 2725, 6764]),
 array([   2,   16,   42, ..., 2199, 2598, 8846]),
 array([    4,     7,     3, ...,  8837, 12919,  5004]),
 array([   28,     1,   147, ...,  5853,  4926, 10066]),
 array([  11, 1250,  548, ..., 4030, 3103, 1965]),
 array([   7,   17,    2, ..., 6575, 4470, 6360]),
 array([   33,     1,     0, ..., 10212, 19401,  4356]),
 array([   7,   36,    6, ...,  439, 7670, 8943]),
 array([   7,    4,   16, ..., 6678, 2725, 6764]),
 array([   6,   57,    3, ..., 9433, 7088, 2000]),
 array([   1,    8,   27, ..., 1363, 4260, 6001]),
 array([  37,   95,    0, ...,  385, 2640,  857]),
 array([   3,    4,  436, ..., 6334, 8539, 1574]),
 array([   8,

実はあまりうまくいっていない
- 気持ちはわからなくもない
- 一般にTransformerは大量のデータを用いて時間をかけて学習することで精度が良くなることが知られている

In [ ]:
import pandas as pd
tbl = list()
for i, id in enumerate(evalcode):
  tbl.append([evalvcab[i], vocab.lookup_token(evalvcab[i]),
    id[0].tolist(), vocab.lookup_token(id[0]),
    id[1].tolist(), vocab.lookup_token(id[1]),
    id[2].tolist(), vocab.lookup_token(id[2]),
    id[3].tolist(), vocab.lookup_token(id[3]),
    id[4].tolist(), vocab.lookup_token(id[4])])
df = pd.DataFrame(tbl)
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,64,i,11,',1250,think,548,see,769,don,286,do
1,49,have,51,been,8,a,152,since,83,no,1,the
2,2267,enjoyed,19,by,1,the,6,in,3,.,8,a
3,2849,learning,7,to,4,of,16,that,2,",",73,about
4,73,about,27,his,46,her,1,the,8,a,77,him
5,1016,machine,550,guns,651,gun,3,.,2,",",696,yards
6,2849,learning,7,to,4,of,16,that,2,",",73,about
7,137,so,2,",",16,that,42,they,24,it,3,.
8,210,much,4,of,7,to,3,.,2,",",61,more
9,16,that,28,he,1,the,147,could,53,she,24,it


---
> The language of poetry is so dense, so multivalent, that it demands a concentrated act of attention
> — and offers its greatest rewards only to those who reread.
>
> (Ezra Pound)
>
> 詩の言葉はあまりに密集し、様々な意味を持つため、集中的な意識を払うことが求められる。そして、
> 再読をする者にだけ最大の報酬を提供するのである。
>
> (エズラ・パウンド)
---

# 課題(Multi-Head Attention)
次のMulti-Head Attentionのコードを参考に、Single-Head AttentionをMulti-Head Attentionに入れ替えて実行させて制度などの変化を確認しなさい

In [ ]:
def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    attn_logits = torch.matmul(q, k.transpose(-2, -1))
    attn_logits = attn_logits / math.sqrt(d_k)
    if mask is not None:
        attn_logits = attn_logits.masked_fill(mask == 0, -9e15)
    attention = F.softmax(attn_logits, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

class MultiheadAttention(nn.Module):
    def __init__(self, input_dim, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0, "Embedding dimension cannot be equal to the number of heads as modulo."
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.qkv_proj = nn.Linear(input_dim, 3*embed_dim)
        self.o_proj = nn.Linear(embed_dim, embed_dim)
        self._reset_parameters()
    def forward(self, x, mask=None, return_attention=False):
        batch_size, seq_length, _ = x.size()
        qkv = self.qkv_proj(x)
        # Separate Q, K, V from linear output
        qkv = qkv.reshape(batch_size, seq_length, self.num_heads, 3*self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3) # [Batch, Head, SeqLen, Dims]
        q, k, v = qkv.chunk(3, dim=-1)
        # Determine value outputs
        values, attention = scaled_dot_product(q, k, v, mask=mask)
        values = values.permute(0, 2, 1, 3) # [Batch, SeqLen, Head, Dims]
        values = values.reshape(batch_size, seq_length, self.embed_dim)
        o = self.o_proj(values)
        if return_attention:
            return o, attention
        else:
            return o

# 課題(PyTorch Transformerを用いた単語予測)

- 日本語による単語補完にトライしてみよう
